In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('user')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Herding Analysis

In [3]:
def herding_stockMarket(year_start, year_end):
    
    base_query = '''
        DECLARE @LeftTail FLOAT
        SET @LeftTail = (
            SELECT DISTINCT PERCENTILE_CONT(0.05) WITHIN GROUP(ORDER BY Rm) OVER () FROM MarketFactors
        )

        DECLARE @RightTail FLOAT
        SET @RightTail = (
            SELECT DISTINCT PERCENTILE_CONT(0.95) WITHIN GROUP(ORDER BY Rm) OVER () FROM MarketFactors
        )

        SELECT
            Datadate,
            MktRf, Smb, Hml, Rmw, Cma, Rf, Mom, Rm, 
            ABS(Rm) AS AbsoluteRm,
            POWER(Rm, 2) AS SquaredRm, 
            Sum / Count AS Csad,
            CASE 
                WHEN Rm <= @LeftTail THEN 1
                ELSE 0
            END AS LeftTail,
            CASE 
                WHEN Rm >= @RightTail THEN 1
                ELSE 0
            END AS RightTail
        FROM (
            SELECT
                DataDate, 
                MktRf, Smb, Hml, Rmw, Cma, Rf, Mom, Rm,
                ABS(SUM(DailyReturns) - Rm) AS Sum,
                COUNT(DISTINCT LPermNo) AS Count
            FROM (
                SELECT
                    A.LPermNo,
                    A.DataDate,
                    B.*,
                    (PrcCd / LAG(A.PrcCd)  OVER (PARTITION BY A.LPermNo ORDER BY A.DataDate)) - 1 AS DailyReturns
                FROM
                    CrspcSecuritiesDaily A
                INNER JOIN
                    MarketFactors B ON A.DataDate = B.Date
                WHERE
                    PrcCd <> 0
                    AND B.Date >= '{}-01-1' AND B.Date < '{}-01-01'
            ) A
            GROUP BY
                DataDate,
                MktRf, Smb, Hml, Rmw, Cma, Rf, Mom, Rm
        ) B
        ORDER BY
            DataDate
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [4]:
start, end = 2002, 2022

df = herding_stockMarket(year_start=start, year_end=end)
df.to_csv('data/output/stocks/herding_stockMarket_{}-{}.csv'.format(start, end), index=False)
df.head()

,Datadate,MktRf,Smb,Hml,Rmw,Cma,Rf,Mom,Rm,AbsoluteRm,SquaredRm,Csad,LeftTail,RightTail
0,2002-01-02,0.0042,-0.0072,0.0017,-0.0026,-0.0041,0.00007,-0.0097,0.00427,0.00427,0.000018,NaN,0,0
1,2002-01-03,0.0099,0.0077,-0.0040,-0.0030,-0.0080,0.00007,-0.0043,0.00997,0.00997,0.000099,0.012651,0,0
2,2002-01-04,0.0070,0.0020,0.0039,-0.0008,-0.0023,0.00007,-0.0029,0.00707,0.00707,0.000050,0.009823,0,0
3,2002-01-07,-0.0070,-0.0024,0.0085,-0.0007,0.0016,0.00007,0.0036,-0.00693,0.00693,0.000048,0.001171,0,0
4,2002-01-08,-0.0023,0.0121,0.0029,-0.0021,-0.0001,0.00007,0.0035,-0.00223,0.00223,0.000005,0.005999,0,0


### Auxiliary Measures

In [5]:
def herding_auxDistinctCusips(year_start, year_end):
    
    base_query = '''
        SELECT
            COUNT(DISTINCT LPermNo)
        FROM
            CrspcSecuritiesDaily
        WHERE
            DataDate >= '{}-01-1' AND DataDate < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [6]:
start, end = 2002, 2022

df = herding_auxDistinctCusips(year_start=start, year_end=end)
df.to_csv('data/output/stocks/herding_auxDistinctCusips_{}-{}.csv'.format(start, end), index=False)
df.head()

,
0,8891


In [7]:
def herding_auxDistinctCusips_by_exchange(year_start, year_end):
    
    base_query = '''
        SELECT
            Exchange,
            COUNT(DISTINCT LPermNo)
        FROM
            CrspcSecuritiesDaily
        WHERE
            DataDate >= '{}-01-1' AND DataDate < '{}-01-01'
        GROUP BY
            Exchange
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [8]:
start, end = 2002, 2022

df = herding_auxDistinctCusips_by_exchange(year_start=start, year_end=end)
df.to_csv('data/output/stocks/herding_auxDistinctCusips_by_exchange_{}-{}.csv'.format(start, end), index=False)
df.head()

,Exchange,
0,NASDAQ,5550
1,NYSE,2940
2,AMEX,436


In [9]:
def herding_auxDistinctCusips_by_industry(year_start, year_end):
    
    base_query = '''
        SELECT
            Industry,
            COUNT(DISTINCT LPermNo)
        FROM
            CrspcSecuritiesDaily
        WHERE
            DataDate >= '{}-01-1' AND DataDate < '{}-01-01'
        GROUP BY
            Industry
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [10]:
start, end = 2002, 2022

df = herding_auxDistinctCusips_by_industry(year_start=start, year_end=end)
df.to_csv('data/output/stocks/herding_auxDistinctCusips_by_industry_{}-{}.csv'.format(start, end), index=False)
df.head()

,Industry,
0,Mining,293
1,Public,564
2,Retail,422
3,Services,1694
4,Missing,16
